In [1]:
import os  
import pandas as pd
import numpy as np
import gensim
from tqdm import tqdm

C:\ProgramData\Anaconda2\lib\site-packages\gensim-1.0.1-py2.7-win-amd64.egg\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
No handlers could be found for logger "gensim.models.doc2vec"


In [2]:
# 载入数据
df = pd.read_csv("D:\Program\PycharmProjects\quoraDupli\data\quora_duplicate_questions.tsv",delimiter='\t')
# df = pd.read_csv("/home/zydar/software/data/PycharmProjects/quoraDupli/data/quora_duplicate_questions.tsv",delimiter='\t')
# 统计
print("number of rows (question pairs): %i"%(df.shape[0]))
print(df['is_duplicate'].value_counts())
unique_qids = set(list(df['qid2'].unique()) + list(df['qid1'].unique()))
print("number of unique questions: %i" % (len(unique_qids)))

# 转化编码，避免NaN
df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))

number of rows (question pairs): 404290
0    255027
1    149263
Name: is_duplicate, dtype: int64
number of unique questions: 537933


## TFIDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))
del questions

In [4]:
import spacy
import en_core_web_md
nlp = en_core_web_md.load()

vecs1 = []
for qu in tqdm(list(df['question1'])):
    doc = nlp(qu) 
    mean_vec = np.zeros([len(doc), 300])
    for word in doc:
        # word2vec
        vec = word.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec += vec * idf
    mean_vec = mean_vec.mean(axis=0)
    vecs1.append(mean_vec)
df['q1_feats'] = list(vecs1)

100%|█████████████████████████████████████████████████████████████████████████| 404290/404290 [22:41<00:00, 296.94it/s]


In [13]:
vecs2 = []
for qu in list(df['question2']):
    doc = nlp(qu) 
    mean_vec = np.zeros([len(doc), 300])
    for word in doc:
        # word2vec
        vec = word.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec += vec * idf
    mean_vec = mean_vec.mean(axis=0)
    vecs2.append(mean_vec)
df['q2_feats'] = list(vecs2)

In [15]:
df.to_json('D:\Program\PycharmProjects\quoraDupli\quora2Vec.json')

In [16]:
df

,id,qid1,qid2,question1,question2,is_duplicate,q1_feats,q2_feats
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[-5.85687178373, 17.4495589584, 4.86272039078,...","[0.398578986526, 13.9916072339, -0.50456432439..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[9.35610295832, 13.098565504, 18.9450982418, -...","[4.64968751371, 9.9749275744, 20.330103159, -0..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[0.909520372748, 16.0502993912, -8.12685599923...","[-17.3051046431, 17.3556136834, -9.13566353917..."
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[-4.95074508945, 17.0988737345, -15.4749648869...","[3.89791050926, 2.54585705698, -2.05379233696,..."
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[-11.5203021914, 19.7699476928, -4.51099687815...","[-5.3912062794, 1.76722100377, 1.81012800336, ..."
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,"[9.25545094907, 28.1794142276, -7.31555020809,...","[6.39677847922, 29.579611674, 5.07470333576, 2..."
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,"[-0.850355960429, -0.474234554917, -7.45004165...","[-12.1050005332, 12.2044582367, -8.76824180782..."
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,"[-0.930315464735, -3.27968811989, -5.318502187...","[2.27953135595, -2.16342492402, -5.58700090647..."
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,"[0.552677646279, 1.84929458797, -7.71049585938...","[0.0130944997072, 2.04112312198, -8.4431048333..."
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0,"[13.5067274049, -0.81839068234, -5.79392570257...","[-11.1235443205, 3.00825281441, -1.47134649754..."
